# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read [the following instructions](https://github.com/sdobnik/computational-semantics/blob/master/README.md) on how to work on group assignments.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for phrases affect both the model and the learned information about similarities.  

Note that this lab uses a code from `dist_erk.py`, which contains functions that highly resemble those shown during the lecture. In the end, you can use either of the functions (from the lecture / from the file) to solve the tasks.

In [1]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus which you can download from [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (wikipedia.txt.zip). (This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/)).  
When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below.  
<!-- <It may already exist in `/opt/mlt/courses/cl2015/a5`.> -->


In [4]:
corpus_dir = './wikipedia/'

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py` earlier, and they largely resemble functions shown during the lecture.

In [5]:
numdims = 1000 # Number of dimensions: number of vectors in a vector space for one word
svddim = 5 # ???

# which words to use as targets and context words?
# we need to count the words and keep only the N most frequent ones
# which function would you use here with which variable?
ktw = do_word_count(corpus_dir,numdims)

wi = make_word_index(ktw) # word index
words_in_order = sorted(wi.keys(), key=lambda w:wi[w]) # sorted words


# create different spaces (the original matrix space, the ppmi space, the svd space)
# which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir, wi, numdims)
print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k, wi)
print('svd transform')
svdspace_1k = svd_transform(ppmispace_1k, numdims, svddim) 
print('done.')

reading file wikipedia.txt
create count matrices
reading file wikipedia.txt
ppmi transform
svd transform
done.


In [6]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])
print('life:', ppmispace_1k['life'])
print('nature:', svdspace_1k['nature'])

house: [2554 3774 3105  567  962  631  443  185  311  189  131   28   93  169
   81  125  151  408  194   90   79   29  217  184   62   15   31   70
   10    1   41   21    1   31   37    1   30    5   25    7    3   20
   11    1   32   36    2    5   66    4    0   46    8   18   28    0
   20    7    8   16   10   40    0  175   10    2    7   19    1  174
   11    3    1    6    0    0    0   10    9   11    7   24    4    4
   14   23   58    7    0   10    2    3   10    6   18    6   13    3
   22    0    3    5    3    7   14    3   40   20   19   15    6    8
   24    4    5    1   19    0    3    1    0   14    0   14   53    7
    7   11    6    5    5    4   12    6   53    1    1  433    4    0
    5    7    7   12    1    1    3    4   17    8   16    1    2   31
    1   12   14    1   44    6   14    9   38    7    2    6    8    1
   10    6   10    1    9    7    9    4    3   10    0   11    3    2
    0    2   11   37    2    0    2    1    5    9   10   16   88    6

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. We saved all three matrices [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (pretrained.zip). Download them and unpack them to a `pretrained` folder which should be a subfolder of the folder with this notebook:

In [7]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load('./pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load('./pretrained/raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load('./pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load('./pretrained/svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [8]:
# testing semantic space
print('house:', space_10k['house'])
print('life:', space_10k['life'])
print('historian:', space_10k['historian'])

house: [2554 3774 3105 ...    0    0    0]
life: [4502 2079 3365 ...    0    0    0]
historian: [693 183  77 ...   0   0   0]


## 3. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture (the one from [2]). For more information, please read the papers.

The following code will transform similarity scores into a Python-friendly format:

In [9]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw))))
print('number of available word pairs to test:', len(word_pairs))
#list(zip(word_pairs, visual_similarity, semantic_similarity))

number of available words to test: 12
number of available word pairs to test: 774


Now we are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [10]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [11]:
from scipy.spatial import distance
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html

raw_similarities  = [1- distance.cosine(space_10k[w1], space_10k[w2]) for w1, w2 in word_pairs]
ppmi_similarities = [1- distance.cosine(ppmispace_10k[w1], ppmispace_10k[w2]) for w1, w2 in word_pairs]
svd_similarities  = [1- distance.cosine(svdspace_10k[w1], svdspace_10k[w2]) for w1, w2 in word_pairs]
#Notes around the web: "spatial.distance.cosine computes the distance, and not the similarity --> subtract the value from 1 to get the similarity"
# I thought cosine distance shows the similarity directly (without subtraction)
# 
#https://en.wikipedia.org/wiki/Cosine_similarity


Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [12]:

rho1, pval1 = stats.spearmanr(semantic_similarity, raw_similarities)
print("""Raw similarities vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho1, pval1))

rho2, pval2 = stats.spearmanr(semantic_similarity, ppmi_similarities)
print("""Ppmi similarities vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho2, pval2))

rho3, pval3 = stats.spearmanr(semantic_similarity, svd_similarities)
print("""SVD similarities vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho3, pval3))

Raw similarities vs. Semantic Similarity:
rho     = 0.1522
p-value = 0.0000
Ppmi similarities vs. Semantic Similarity:
rho     = 0.4547
p-value = 0.0000
SVD similarities vs. Semantic Similarity:
rho     = 0.4232
p-value = 0.0000


Here, Pearson's correlation coefficient is calculated based on the lists of similarity scores I calculated in [54]. (similarly to code in [42])
Since the greater the rho number, the better the similarity scores align, it seems that the ppmi model model works best. The coefficient is statistically significant in all models since it's 0 (so less than 0.05).

I am not quite sure what I should expect :| 


We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [13]:
rho4, pval4 = stats.spearmanr(visual_similarity, raw_similarities)
print("""Raw similarities vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho4, pval4))

rho5, pval5 = stats.spearmanr(visual_similarity, ppmi_similarities)
print("""Ppmi similarities vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho5, pval5))

rho6, pval6 = stats.spearmanr(visual_similarity, svd_similarities)
print("""SVD similarities vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho6, pval6))

Raw similarities vs. Visual Similarity:
rho     = 0.1212
p-value = 0.0007
Ppmi similarities vs. Visual Similarity:
rho     = 0.3838
p-value = 0.0000
SVD similarities vs. Visual Similarity:
rho     = 0.3097
p-value = 0.0000


Again the PPMI model works best but all models are less efficient than before. The coefficients remain the same, apart from that in the raw model, which means that it is less significant than the rest, but I am not aware why this is the case

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions. For example, we can subtract the normalised vectors for `king` minus `queen` and add the resulting vector to `man` and we hope to get the vector for `woman`. Why? **[3 marks]**

We expect that "man" is closer in the vector space to "king" and "woman" is closer to "queen". Or, man/woman and king/queen are diametrically opposite. However, this is a case that probably fits more in human intuition rather algebra as such. These words however can co-occur often, so they are likely to be similar.  

Here is some helpful code that allows us to calculate such comparisons.

In [14]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [15]:
short = normalize(svdspace_10k['short'])
light = normalize(svdspace_10k['light'])
long = normalize(svdspace_10k['long'])
heavy = normalize(svdspace_10k['heavy'])

find_similar_to(light - (heavy - long), svdspace_10k)[:10]

[('long', 0.8733111261346901),
 ('above', 0.8259671977311955),
 ('around', 0.8030776291120685),
 ('sun', 0.7692439111243973),
 ('just', 0.7678481974778111),
 ('wide', 0.767257431992253),
 ('each', 0.7665960260861158),
 ('circle', 0.7647746702909336),
 ('length', 0.7601066921319761),
 ('almost', 0.7542351860536628)]

Probably we expect something close to the "short" meaning. I have no idea on the "sun" semantic representation and the "circle" as well. "Sun" and "circle" maybe corellate because the are both round (?) I think most results are metaphor interpretations, which are not incorretct, I was sure that at least one of the results would be the word "tall", which is the most obvious literal analogy for short.

Find 5 similar pairs of pairs of words and test them. Hint: Google for `word analogies examples`. You can also construct analogies that are less lexical but more grammatical, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) from [3]. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [16]:
large = normalize(svdspace_10k['large'])
small = normalize(svdspace_10k['small'])
full = normalize(svdspace_10k['full'])
empty = normalize(svdspace_10k['empty'])
sun = normalize(svdspace_10k['sun'])
moon = normalize(svdspace_10k['moon'])
rich = normalize(svdspace_10k['rich'])
poor = normalize(svdspace_10k['poor'])
day = normalize(svdspace_10k['day'])
night = normalize(svdspace_10k['night'])


find_similar_to(sun - (moon - night), svdspace_10k)[:10]

[('night', 0.8886038339830423),
 ('morning', 0.8000957226947445),
 ('sun', 0.777799161109718),
 ('day', 0.7647056301917052),
 ('evening', 0.7575688171019718),
 ('watching', 0.75365192064434),
 ('sunday', 0.7464716284374743),
 ('end', 0.7453067996593553),
 ('room', 0.744111900775791),
 ('place', 0.7433668475829194)]

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to look at how different semantic composition models, introduced in [2] correlate with human judgements. The file with the dataset is `mitchell_lapata_acl08.txt` included with this notebook. Your task is to do the following:  

(i) process the dataset, extract pairs of `reference - landmark high` and `reference - landmark low`; you can use the code from the lecture as something to start with. Note that there are 2 landmarks for each reference: one landmark exhibits high similarity with the reference, while another one has low similarity with the reference. A single human participant could have evaluated both of these pairs. For more details, we refer you to the paper.  

(ii) build models of semantic phrase composition: in the lecture we introduced simple additive, simple multiplicative and combined models (details are in [2]). Your task is to take a single pair (a reference or a high similarity landmark or a low similarity landmark) and compute the composition of its vectors using each of these functions. Thus, you will have three compositional models that take a `noun - verb` pair and output a single vector, representing the meaning of this pair. As your semantic space, you can use pretrained spaces (standard space, ppmi or svd) introduced above. It is up to you which space you use, but for someone who runs your code, it should be pretty straightforward to switch between them.

(iii) calculate Spearman correlation between each model's predictions and human judgements; you should have something similar to the scores that are shown in the paper [2]:  

![title](./res.png)

The paper states that they calculated correlations between each individual participant's judgeements and each model's predictions.  

Let's say we have 3 models: simple additive (A), simple multiplicative (M), combined (C).  
From our task dataset, we also know that we have 20 participants.  
Now, for each participant in 20 participants we get all `verb - noun` pairs that these participated evaluated.  
For example:

In [17]:
participant_judgemenets_example = [
 'participant50 chatter child gabble 6 high',
 'participant50 chatter tooth click 2 high',
 'participant50 reel head whirl 5 high',
 'participant50 reel mind stagger 4 low',
 'participant50 reel industry stagger 5 high',
 'participant50 reel man whirl 3 low',
 'participant50 glow fire beam 7 low',
 'participant50 glow face burn 3 low',
 'participant50 glow cigar burn 5 high',
 'participant50 glow skin beam 7 high'
    
]

In [18]:
participant_judgemenets_example

['participant50 chatter child gabble 6 high',
 'participant50 chatter tooth click 2 high',
 'participant50 reel head whirl 5 high',
 'participant50 reel mind stagger 4 low',
 'participant50 reel industry stagger 5 high',
 'participant50 reel man whirl 3 low',
 'participant50 glow fire beam 7 low',
 'participant50 glow face burn 3 low',
 'participant50 glow cigar burn 5 high',
 'participant50 glow skin beam 7 high']

Let's look at the first pair that participant50 evaluated: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. The human gave the similarity score of 6 (very similar). Thus, human similarity judgment = [6].  

Our A model's output:  
cosine(p1, p2) = 0.88, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  

Therefore, we have human rating vector [6] and model A output [0.88]. Next is to compute correlation between these two vectors.

To get an overall score, simply average your correlation scores over all participants, since you are calculating correlation scores per participant.

Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]) where each element is a participant's judgement of a specific pair. Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.

The minimum to do in this task: compute correlations for 3 models mentioned above and human rating for AT LEAST one participant. Elaborate on how different the resulting correlation scores are depending on the model's composition function (additive, multiplicative, combined). For examples on how to interpret the results, look at Section 5 Results of the original paper.

In [116]:
# i) process dataset

with open('./mitchell_lapata_acl08.txt', 'r') as file:
    dataset = file.readlines()

for_model = []

human_judg = []


def judgements(participant):
    
    for line in dataset:
        if line.startswith(participant):
            x = line.strip().split(' ')
            participant = x[0]
            ref1 = x[2],x[1]
            ref2 = x[2],x[3]
            in_put = x[4]
            hilo = x[5]
            if ref1[0] in ktw_10k and ref1[1] in ktw_10k and ref2[1] in ktw_10k: #if noun - verb do not exist in the words used to create ppmi model, it will raise an error in next function
                for_model.append([participant, tuple(ref1), tuple(ref2), in_put, hilo])
                
                
    for element in for_model: #find the human average for this participant's phrases
        human = 0
        index = 0
        for line in dataset:
            if element[1][0] in line and element[1][1] in line and element[2][1] in line :
                x = line.strip().split(' ')
                human += int(x[4])
                index += 1
        human_judg.append(human/index)

        
        
        
    for i in for_model:
        print(i)

judgements("participant20")

print(human_judg)

['participant20', ('butler', 'bow'), ('butler', 'submit'), '3', 'low']
['participant20', ('company', 'bow'), ('company', 'submit'), '5', 'high']
['participant20', ('sale', 'boom'), ('sale', 'thunder'), '3', 'low']
['participant20', ('gun', 'boom'), ('gun', 'thunder'), '6', 'high']
[2.9411764705882355, 4.323529411764706, 2.7941176470588234, 5.617647058823529]


In [117]:
# ii) create Additive, Multiplicative, Combined models
    

add_cos = []
mult_cos = []
comb_cos = []



def additive(noun_vector,verb_vector,landmark_vector):
    add1_pair = noun_vector + verb_vector
    add2_pair = noun_vector + landmark_vector
    add_cos.append(1- distance.cosine(add1_pair, add2_pair)) #cosine value
    #return add1_pair, add2_pair

def multiplicative(noun_vector,verb_vector,landmark_vector):
    mult1_pair = noun_vector * verb_vector
    mult2_pair = noun_vector * landmark_vector
    mult_cos.append(1- distance.cosine(mult1_pair, mult2_pair)) #cosine value
    #return mult1_pair, mult2_pair

def combined(noun_vector, verb_vector, landmark_vector, alpha=0, beta=0.95, theta=0.05):
    comb1_add_pair = (alpha * noun_vector) + (beta * verb_vector)
    comb2_add_pair = (alpha * noun_vector) + (beta * landmark_vector)
    comb1_mult_pair = theta * noun_vector * verb_vector
    comb2_mult_pair = theta * noun_vector * landmark_vector
    p_reference = comb1_add_pair + comb1_mult_pair
    p_landmark = comb2_add_pair + comb2_mult_pair
    comb_cos.append(1- distance.cosine(p_reference, p_landmark)) #cosine value
    #return p_reference, p_landmark

#I used the ppmi space because it presented the best results in the previous task

space = ppmispace_10k
 
# In order to find how similar 2 phrases are, we need to find the vectors of their components (discussion-thrive-digress)
# and then combine those vectors

for element in for_model: #feed the model with specific words retrieved from the dataset list
    noun_vector = space[element[1][0]] # find vector for the noun
    verb_vector = space[element[1][1]] # find vector for the verb1
    landmark_vector = space[element[2][1]] # find vector for the verb2
    hilo = element[4] 
    additive(noun_vector,verb_vector,landmark_vector) # add model
    multiplicative(noun_vector,verb_vector,landmark_vector) # mult model
    combined(noun_vector, verb_vector, landmark_vector, alpha=0, beta=0.95, theta=0.05) # comb model

print(for_model)
print(add_cos)
print(mult_cos)
print(comb_cos)

[['participant20', ('butler', 'bow'), ('butler', 'submit'), '3', 'low'], ['participant20', ('company', 'bow'), ('company', 'submit'), '5', 'high'], ['participant20', ('sale', 'boom'), ('sale', 'thunder'), '3', 'low'], ['participant20', ('gun', 'boom'), ('gun', 'thunder'), '6', 'high']]
[0.5526327683917539, 0.6583911183944487, 0.5523089123061238, 0.6361155776620226]
[0.008834018472734573, 0.02816092957418237, 0.002198703885207931, 0.022650956401095823]
[0.05569285328761442, 0.05625826421678992, 0.04920854106442152, 0.04945030577212095]


In [118]:
#iii) find models' correlations with human judgement

rho_add, pval_add = stats.spearmanr(add_cos, human_judg)
print("""Additive model vs. Human judgement:
rho     = {:.4f}
p-value = {:.4f}""".format(rho_add, pval_add))

rho_mult, pval_mult = stats.spearmanr(mult_cos, human_judg)
print("""Multiplicative model vs. Human judgement:
rho     = {:.4f}
p-value = {:.4f}""".format(rho_mult, pval_mult))

rho_comb, pval_comb = stats.spearmanr(comb_cos, human_judg)
print("""Combined model vs. Human judgement:
rho     = {:.4f}
p-value = {:.4f}""".format(rho_comb, pval_comb))

Additive model vs. Human judgement:
rho     = 0.8000
p-value = 0.2000
Additive model vs. Human judgement:
rho     = 0.8000
p-value = 0.2000
Additive model vs. Human judgement:
rho     = 0.4000
p-value = 0.6000


**Any comments/thoughts should go here:**

# Literature

  - [1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

  - [2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
  - [3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

## Marks

This assignment has a total of 60 marks.